In [1]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

#BATCH_SIZE = 100
#WEIGHT_DECAY = 0.00005
#LEARNING_RATE = 0.0001
#EPOCH = 100
PATH = os.getcwd()
#TRAINPATH = "C:\\Users\\arimoto\\cnntest\\train"
#TESTPATH = "C:\\Users\\arimoto\\cnntest\\test"
#VALIDPATH = "C:\\Users\\arimoto\\cnntest\\valid"

#手指姿勢CSVファイル一覧取得
dataset_dir = PATH + "\\dataset"

In [2]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data, adrs, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = data
        self.adrs = adrs
        
        self.datanum = data.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_adrs = self.adrs[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_data = np.array(i_data.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)
        #print(out_data.shape)
        #print(out_label.shape)
        #print(out_mask.shape)
        #print(out_lmask.shape)
        
        return out_data, i_adrs
        #return batch_set

In [3]:
class Ignore(nn.Module):
    def __init__(self, input_dim, output_dim, activation = lambda x: x):
        '''
        引数:
            input_dim: 入力次元
            output_dim: 出力次元
            activation: 活性化関数
        パラメータ:
            W: 重み
            b: バイアス
        '''
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(np.random.normal(size = (output_dim, input_dim))))
        self.b = nn.Parameter(torch.Tensor(np.zeros(output_dim)))
        self.activation = activation
    def forward(self, x, mask):
        
        out = torch.empty_like(x).to(torch.device("cuda:0"))
        
        masked_x = torch.mul(x, mask)
        
        try:
            m_size = torch.Tensor(mask.size()[1]).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask, 1).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            for b in range(out.size()[0]):
                out[b] = torch.add(torch.mul(rate[b], torch.sum(torch.mul(masked_x[b], self.W), 1)), self.b)
        except IndexError:
            m_size = torch.Tensor(mask.size()).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            out = torch.add(torch.mul(rate, torch.sum(torch.mul(masked_x, self.W))), self.b)
        
        return self.activation(out)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        #CNN
        self.pool = nn.AvgPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)
        self.bn2d1 = nn.BatchNorm2d(16)
        self.bn2d2 = nn.BatchNorm2d(32)
        
        self.fcm = Ignore(32 * 7 * 7 + 43, 32 * 7 * 7 + 43)
        self.fcm_c = Ignore(32 * 7 * 7 + 64, 32 * 7 * 7 + 64)
        self.bnm = nn.BatchNorm1d(32 * 7 * 7 + 43)
        self.bnm_c = nn.BatchNorm1d(32 * 7 * 7 + 64)
        #fully connect for hand Location(x,y,z)
        self.fcL1 = nn.Linear(32 * 7 * 7 + 43, 300)
        self.fcL1_c = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcL2 = nn.Linear(300, 30)
        self.fcL3 = nn.Linear(30, 3)        
        self.bnL1 = nn.BatchNorm1d(300)
        self.bnL2 = nn.BatchNorm1d(30)
        #fully connect for hand Pose Descriptor(8 properties)
        self.fcPD1 = nn.Linear(32 * 7 * 7 + 43, 300)
        self.fcPD1_c = nn.Linear(32 * 7 * 7 + 64, 300)
        self.fcPD2 = nn.Linear(300, 60)
        self.fcPD3 = nn.Linear(60, 8)
        self.bnPD1 = nn.BatchNorm1d(300)
        self.bnPD2 = nn.BatchNorm1d(60)
        
        #Autoencoder
        self.mask = Ignore(63, 63)#self.mask = Ignore(84, 84)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dense_enc1 = nn.Linear(63, 32)#self.dense_enc1 = nn.Linear(84, 40)
        self.bn1 = nn.BatchNorm1d(32)#(40)
        self.dense_enc2 = nn.Linear(32, 16)#self.bn1 = nn.BatchNorm1d(40, 18)
        self.bn2 = nn.BatchNorm1d(16)#self.bn2 = nn.BatchNorm1d(18)
        self.dense_enc3 = nn.Linear(16,8)#self.dense_enc3 = nn.Linear(18, 8)
    
        self.dense_dec1 = nn.Linear(8,16)
        self.bn4 = nn.BatchNorm1d(16)
        self.dense_dec2 = nn.Linear(16, 26)#self.dense_dec2 = nn.Linear(16, 32)
        self.bn5 = nn.BatchNorm1d(26)#self.bn5 = nn.BatchNorm1d(32)
        self.drop1 = nn.Dropout(p=0.2)
        self.dense_dec3 = nn.Linear(26, 42)#self.dense_dec3 = nn.Linear(32, 63)

    def encoder(self, x):
        #x = torch.div(x, 100.)
        #x = self.mask(x, m)
        x = self.dense_enc1(x)
        x = self.bn1(self.relu(x))
        x = self.dense_enc2(x)
        x = self.bn2(self.relu(x))
        x = self.dense_enc3(x)
        return x

    def decoder(self, x):
        x = self.tanh(x)
        x = self.dense_dec1(x)
        x = self.bn4(self.relu(x))
        x = self.dense_dec2(x)
        x = self.bn5(self.relu(x))
        #x = self.drop1(x)
        x = self.dense_dec3(x)
        #x = self.mask(x, m)
        #x = torch.mul(x, 100.)
        return x

    def forward(self, x=None, z=None):
        if x != None and z == None:
            z = self.encoder(x)
            x = self.decoder(z)
        elif x == None and z != None:
            print("decoder only")
            x = self.decoder(z)
        return x, z


In [4]:
#出力値保存
def outcsv(n_epoch, eval_input_value, eval_output_value, eval_mid_value, adrs):
    for b in range(len(eval_input_value)):
        fname = f"dataset_hand_mm_{adrs[b][0]}_{adrs[b][1]}_{adrs[b][2]:03}_{adrs[b][3]:08}_{adrs[b][4]:04}.csv"
        os.makedirs(PATH + "\\outputs", exist_ok=True)
        os.makedirs(PATH + "\\outputs\\inputs", exist_ok=True)
        os.makedirs(PATH + "\\outputs\\outputs", exist_ok=True)
        os.makedirs(PATH + "\\outputs\\mid", exist_ok=True)
        os.makedirs(PATH + "\\outputs\\img", exist_ok = True)
        os.makedirs(PATH + "\\outputs\\img\\input", exist_ok = True)
        os.makedirs(PATH + "\\outputs\\img\\output", exist_ok = True)
        print(fname)
        with open(PATH + "\\outputs\\inputs\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(eval_input_value[b])
        with open(PATH + "\\outputs\\outputs\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(eval_output_value[b])
        with open(PATH + "\\outputs\\mid\\" + fname, mode='w') as f:
            writer = csv.writer(f)
            writer.writerow(eval_mid_value[b])
    print("fin save.")
    return

def saveloss(train, test):
    with open(PATH + "\\outputs\\loss_values.csv", mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([train, test])

In [25]:
csvlist = glob.glob(dataset_dir + "\\*.csv")
#print(csvlist)
data_sets = []
inputs = []
valid_data_num = 0
invalid_data_num = 0
valid_label_num = 0
invalid_label_num = 0
for i in range(len(csvlist)):
    points = []
    date = []
    file_name = csvlist[i]
    #dataname = file_name[-51:]
    file_name = file_name.split(dataset_dir + "\\dataset_hand_mm_")
    file_name = file_name[1].rsplit(".csv")
    file_name = file_name[0].split("_")
    date.append(int(file_name[0]))
    date.append(int(file_name[1]))
    date.append(int(file_name[2]))
    date.append(int(file_name[4]))
    date.append(int(file_name[5]))
    #date.append(int(file_name[-35:-27]))
    #date.append(int(file_name[-26:-20]))
    #date.append(int(file_name[-19:-18]))
    #date.append(int(file_name[-17:-9]))
    #date.append(int(file_name[-8:-4]))
    print(date)
    with open(csvlist[i]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    if float(point) < -5000.:
                        points.append(float(0))
                    else:
                        points.append(float(point))
        points = np.asarray(points)
        num += 1
    data_pair = []
    data_pair.append(points)
    data_pair.append(date)
    data_sets.append(data_pair)

print("fin loading.")
data_sets = np.asarray(data_sets)
#print("data" + str(data_sets[0]))
#print("data", data_label_sets[0][0])
#print("label" + str(label_sets[0]))
#print("label", data_label_sets[1][0])
#print(type(data_sets[0][0]))
#print(type(label_sets[0][0]))
print("data_sets", data_sets.shape)
data_eval = []
adrs_eval = []

for i in range(len(data_sets)):
    data_eval.append(data_sets[i][0])
    adrs_eval.append(data_sets[i][1])
print(data_eval[0])
print(adrs_eval[0])
data_evalF = np.array(data_eval).astype('float32')/100
adrs_evalF = np.array(adrs_eval)

#adrs_evalF = adrs_evalF[:, np.newaxis]

print("data          train ",data_evalF.shape)
print("address       train ",adrs_evalF.shape)

BATCH_SIZE = data_evalF.shape[0]
print(BATCH_SIZE)
evalset = Mydatasets(data = data_evalF, adrs = adrs_evalF,)
evalloader = torch.utils.data.DataLoader(evalset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

device = torch.device("cuda:0")
device_cpu = torch.device("cpu")
net = Autoencoder()
net.load_state_dict(torch.load(PATH + "\\hand_AE_model"))
net = net.to(device)
for p in net.parameters():
    p.requires_grad = False
    
eval_mid_value = [] #trainingの中間層の出力を保持するlist
eval_input_value = []   #trainingの入力を保持するlist
eval_output_value = []  #trainingの出力を保持するlist
eval_adrs = []

eval_size = data_evalF.shape[0]

#ここから推定開始
print("eval")
net.eval()
teval = 0
for (inputs, adrs) in evalloader:
    teval += 1
    #物体位置基準座標→手首位置基準座標
    handlocs = np.zeros_like(inputs)
    #print(inputs.size()[0])
    for bsize in range(inputs.size()[0]):
        for i in range(int(inputs.size()[1] / 4)):
            handlocs[bsize][i*4+0] = (inputs*100)[bsize][0].item()
            handlocs[bsize][i*4+1] = (inputs*100)[bsize][1].item()
            handlocs[bsize][i*4+2] = (inputs*100)[bsize][2].item()
    handlocs = torch.from_numpy(handlocs.astype(np.float32)).clone()
    #print("handlocs:", handlocs[0])
    #print("handlocs:",handlocs.shape)
    inputs_h = ((inputs*100 - handlocs)/100).to(device)
    inputs = inputs.to(device)
    #print("inputs_h:",inputs_h.shape)
    #ネットワークから出力を得る
    outputs, mid = net(x = inputs_h)
    
    #手首位置基準座標→物体位置基準座標
    handlocs3d = torch.zeros_like(outputs).to(device)
    #print(outputs.size()[0])
    for bsize in range(outputs.size()[0]):
        for i in range(int(handlocs.size()[1] / 4)):
            handlocs3d[bsize][i*3+0] = handlocs[bsize][i*4+0]
            handlocs3d[bsize][i*3+1] = handlocs[bsize][i*4+1]
            handlocs3d[bsize][i*3+2] = handlocs[bsize][i*4+2]
    outputs_o = (outputs*100 + handlocs3d).to(device)
    #print("handlocs3d:",handlocs3d.shape)
    #print("inputs:",inputs.shape)
    #print("outputs_o:",outputs_o.shape)
    #print("mid:",mid.shape)
    
    eval_input_value = (inputs*100).to(device_cpu).detach().numpy().copy()
    eval_output_value = outputs_o.to(device_cpu).detach().numpy().copy()
    eval_mid_value = mid.to(device_cpu).detach().numpy().copy()
#print(len(eval_input_value))
#print(len(eval_output_value))
#print(len(eval_mid_value))
#print(eval_input_value[0])
#print(eval_output_value[0])
#print(eval_mid_value[0])
outcsv(1, eval_input_value, eval_output_value, eval_mid_value, adrs)
#eval_input_value.clear()
#eval_output_value.clear()
#eval_mid_value.clear()

[20200116, 223630, 45, 9004, 1]
[20200116, 223630, 45, 9005, 2]
[20200116, 223630, 45, 9006, 3]
[20200116, 223630, 45, 9007, 4]
[20200116, 223630, 45, 9008, 5]
[20200116, 223630, 45, 9009, 6]
[20200116, 223630, 45, 9010, 7]
fin loading.
data_sets (7, 2)
[ 4.07740e+02 -1.25199e+02 -1.41000e+02  4.51434e-01  3.76318e+02
 -1.15808e+02 -1.42000e+02  5.91748e-01  3.42703e+02 -1.11767e+02
 -1.38000e+02  6.71248e-01  3.16491e+02 -1.05619e+02 -1.31915e+02
  6.85709e-01  2.90789e+02 -1.05588e+02 -1.25947e+02  4.74967e-01
  3.36738e+02 -1.38540e+02 -1.17000e+02  6.30000e-01  3.15059e+02
 -1.44153e+02 -7.30000e+01  7.03599e-01  3.01896e+02 -1.35344e+02
 -1.42051e+02  7.89231e-01  2.89918e+02 -1.29200e+02 -2.04888e+02
  7.60097e-01  3.47868e+02 -1.39121e+02 -1.31000e+02  5.26194e-01
  3.26323e+02 -1.36934e+02 -9.10000e+01  4.20939e-01  3.24031e+02
 -1.36702e+02 -8.67430e+01  4.09553e-01  3.22336e+02 -1.36530e+02
 -8.35963e+01  3.17688e-01  3.59205e+02 -1.30427e+02 -1.42000e+02
  4.90619e-01  3.418